In [1]:
import sys
import glob
import os
import pandas as pd
import ee
import numpy as np
import geopandas as gp

import matplotlib.pyplot as plt
import rsfuncs as rs

from scipy import stats
from scipy.signal import savgol_filter
from datetime import datetime, timedelta
from datetime import datetime as dt

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [18,16]
ee.Initialize()

In [2]:
# EE functions are in the rsfuncs module. Loacal functions are here: 

In [3]:
def read_sm_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def get_ims_by_date(ims_list, var, res=10):
    imlist = []
    imdates = []
    num_images = len(ims_list.getInfo())

    for i in range (0, num_images):
        
        if i % 5 == 0:
            print(str((i / num_images)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims_list.get(i))
        imlist.append(array_from_latlon(latlon, var, res))
        date =  latlon.get('system:time_start')
        info_dict = ims.get(i).getInfo()
        date = info_dict['id']
        imdates.append(date)

    return imlist, imdates

def array_from_latlon(latlon_obj, var, res ):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=res)
    try:
        lats = np.array((ee.Array(res.get("latitude")).getInfo()))
        lons = np.array((ee.Array(res.get("longitude")).getInfo()))
        data = np.array((ee.Array(res.get(var)).getInfo()))
    except:
        data = np.full_like(lats, np.nan,dtype=np.float64)
    
    out = make_np_array(data, lats, lons)
    return out   

def make_np_array(data, lats, lons):
    # get data from df as arrays
    lons = np.array(lons)
    lats = np.array(lats)
    data = np.array(data) # Set var here 
                                              
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    
    return arr

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

def get_ndvi(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("NDVI")).getInfo()))
    out = np.array(data)
    return out 

def round_up_to_odd(f):
    f = int(np.ceil(f))
    return f + 1 if f % 2 == 0 else f

In [4]:
site_file = gp.read_file("../shape/scan_sites.shp")
sites = site_file[~site_file['state'].isin(["AK", "HI", "PR", "VI"])]

In [5]:
data = rs.load_data()

In [6]:
data_dir = "../data"

In [8]:
out_dict = {}

In [57]:
# For each site id, find the sm file with the data (in data dir), 
# calculate psi as psi = A * SM ^b (krishna's paper )
# query the (1) landcover, (2) Sentinel backscatter (Prism P), MODIS / Landsat LAI for the whole timeseries 
   
for idx, row in sites.iterrows():
    
    if row.id in out_dict.keys():
        print(row.id)
        continue
        
    print("Processings site no {}".format(row.id))
    
    # Make geom to submit to EE 
    x,y = row.geometry.buffer(0.0001).envelope.exterior.coords.xy
    coords = [list(zip(x,y))]
    area = ee.Geometry.Polygon(coords)
    
    # Get the corresponding SCAN data file from data folder
    site_id = row.id
    sm_file = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if site_id in x][0]
    sm_dat = read_sm_file(sm_file)
    sm_dat['Date'] =  pd.to_datetime(sm_dat['Date'], format='%Y%m%d %H:%M')
    sm_dat.set_index('Date', inplace=True)
        
    # start and end date
    if sm_dat.empty:
        print("no valid soil moisture data for {}".format(row.id))
        continue
        
    startdate = sm_dat.index[0]
    enddate = sm_dat.index[-1]
    
    date = startdate.strftime("%Y-%m-%d")
        
    # Select the nlcd dataset
    dataset =  rs.load_data()['nlcd']
    ic = dataset[0]
    var = dataset[1]
    res = dataset[3]
    
    # find the nearest nlcd dataset 
    col = ic.filterDate(ee.Date(date).advance(-1, 'years'), ee.Date(date).advance(2, 'years')).first()
    if not col.getInfo():
        col = ic.filterDate(ee.Date(date).advance(-3, 'years'), ee.Date(date).advance(2, 'years')).first()
    t = col.reduceRegion(ee.Reducer.frequencyHistogram(), area, res).get(var)
    meta = col.getInfo()
    lc_class = int(list(t.getInfo().keys())[0])
    
    # Get the landcover type
    lcidx = meta['properties']['landcover_class_values'].index(lc_class)
    lctype = meta['properties']['landcover_class_names'][lcidx]
    print(lctype)
    
    # Get Sentinel images and dates (ascending orbits only, VV polarization)
    s1 = rs.load_data()['s1']
    s1ic, s1var, s1res = s1[0], s1[1], s1[3]
    
    s1_col = s1ic.filterBounds(area).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    s1_col = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).select(s1var)
    
    # Krishna used ascending pass... I think descending is the correct orbit for the AM 
    
    s1_col = s1_col.filterDate(startdate,enddate)
    t = s1_col.sort('system:time_start')
    
    try:
        ims = t.toList(t.size())
        print("Processing {} sentinel overapasses".format(len(ims.getInfo())))
        s1dat, dates = get_ims_by_date(ims,s1var)
        
    except:
        print("no valid overpasses")
        continue
    
    # Calc the S1 backscatter in each image
    mean_sigmas = []
    std_sigmas = []

    for i in s1dat:
        mean_sigmas.append(np.mean(i))
        std_sigmas.append(np.std(i))
        
    # Convert the datestrings from S1 to pandas datetimes 
    for idx, x in enumerate(dates):
        timestamp = x.find("V_")+2
        timestr = x[timestamp:timestamp+13]
        dates[idx] = pd.to_datetime(timestr, format='%Y%m%d %H:%M')
        
    # Get PRISM data for all the S1 overpass dates to filter the rainy days
    print("processing PRISM")
    rainfall = []

    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
        
        y,m,d = dates[i].year, dates[i].month, dates[i].day
        t = filter_date(rs.load_data()['prism_daily'][0], y, m, d).sum()
        precip_total = get_2day_precip(t, area)
        rainfall.append(precip_total)

    
    # MODIS LAI 
    print("processing MODIS")
    modis = rs.load_data()['modis_lai']
    mic, mvar, mres = modis[0],modis[1], modis[2]

    modis_lai = []
    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
            
        y,m,d = dates[i].year, dates[i].month, dates[i].day

        start = ee.Date.fromYMD(y,m,d).advance(-2,"day")
        end = ee.Date.fromYMD(y,m,d).advance(2, "day")

        prod = mic.filterDate(start, end).sort('system:time_start', False).select(mvar).first()

        try:
            res = prod.reduceRegion(reducer=ee.Reducer.first(),geometry=area,scale=5) # Need to have scale smaller than geom
            data = np.array((ee.Array(res.get(mvar)).getInfo()))
            out = np.array(data)* 0.1
            modis_lai.append(out)
        except:
            out = np.nan
            modis_lai.append(np.nan)
            
    # Landsat - Note: some sites are in the overlap areas between passes.
    # these sites can have multiple obs / day or obs separated by 8days instead of 16. 
    
    print("Processing Landsat")
    landsat = rs.load_data()['l8_sr']
    lic, lvar, lsf = landsat[0],landsat[1], landsat[2]

    lstart = ee.Date.fromYMD(startdate.year,startdate.month,startdate.day).advance(-16,"day")
    lend = ee.Date.fromYMD(enddate.year,enddate.month,enddate.day).advance(16, "day")

    l8_col = lic.filterDate(lstart,lend).filterBounds(area).map(rs.mask_quality) # Mask clouds and shadows 
    lt = l8_col.sort('system:time_start')
    lims = lt.toList(lt.size())

    num_ims = len(lims.getInfo())

    ldfs = []

    for i in range (0, num_ims):
        if i % 5 == 0:
            print(str((i / num_ims)*100)[:5] + " % ")

        ls_latlon = ee.Image.pixelLonLat().addBands(lims.get(i))
        ltemp = ls_latlon.select(["B1","B2","B3","B4","B5","B6","B&"]).multiply(lsf)
        l8_res = ltemp.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,bestEffort=True,scale=30)

        l8_info_dict = lims.get(i).getInfo()
        l8_date = l8_info_dict['id'][-8:]

        l8_out = l8_res.getInfo()

        ldf = pd.DataFrame.from_dict(l8_out.values()).T
        ldf.columns = l8_out.keys()
        ldf.index = pd.to_datetime([l8_date])
        ldfs.append(ldf)
        
                        
    # Filter the SCAN data for the S1 dates, 3 am - 7 am 
    rzsm = []
    ssm = []
    
    for i in dates:
        starttime = i.replace(second=0, microsecond=0, minute=0, hour=3)
        endtime = starttime+timedelta(hours= 4)
        df = pd.DataFrame(sm_dat[starttime:endtime])
        
        rzsm.append(df[df.columns[-1]].values)
        ssm.append(df[df.columns[-3]].values)
    
    # In case there are nans or data gaps in the sm data
    rzsm = [list(filter(None, x)) for x in rzsm]
    ssm = [list(filter(None, x)) for x in ssm]
    
    # Calc the 5 hour mean for each sensor reading 
    for idx,x in enumerate(rzsm):
        rzsm[idx] = np.nanmean([np.float(i) for i in x])
    
    for idx,x in enumerate(ssm):
        ssm[idx] = np.nanmean([np.float(i) for i in x])

    # params to calculate psi 
    a = row.a
    b = row.b
    
    # RZ and Surface LWP = A * sm ^b
    psi1 = [a*(x/100)**b for x in rzsm]
    psi2 = [a*(x/100)**b for x in ssm]
    
    # Make a dataframe out of everything
    df = pd.DataFrame([dates, mean_sigmas, std_sigmas, rainfall, modis_lai, rzsm,psi1,ssm,psi2, [lcidx]*len(dates)])
    df = df.T
    df.columns = (['date', "sigma", "std_sigma", "precip","LAI", "rzsm","psi_rz", "ssm", "psi_s", "lc_type"])
    df = rs.col_to_dt(df) # set the date col as datetime index 
    mask=(df['precip'] < 0.1)
    dfout = df[mask]

    # For now append the nearest landsat image band values to the master df 
    # TODO: Golay interpolation 
    
    ls = pd.concat(ldfs)
    win_len = round_up_to_odd(num_ims/7)
    
    for i in ls.columns:
        ls[i][ls[i] == 0] = np.nan
        ls[i] = ls[i].interpolate(method = "linear")
        ls[i+"_filt"] = savgol_filter(ls[i], window_length=win_len, polyorder=2)
    
    ls_df = ls[(ls.T != 0).any()]
    ls_df = ls_df.groupby(level = 0).mean()
    
    # Break the loop if there are no non- rainy sentinel overpasses 
    if dfout.empty:
        print("No non-rainy overpaasses ")
        continue 
        
    l8_dfs = []
    
    for i in dfout.index:
        l8_idx = ls_df.index.get_loc(i, method='nearest')
        l8_contemp = ls_df.iloc[l8_idx]
        t = pd.DataFrame(l8_contemp).T
        l8_dfs.append(t)
    
    fin_ls = pd.concat(l8_dfs)

    FIN = pd.concat( [dfout.reset_index(drop=True), fin_ls.reset_index(drop=True)], axis=1) 
    FIN.index = dfout.index

    print(FIN.head())
    
    out_dict[(row.id)] = FIN

2057
2078
2177
2113
2174
2055
2173
2180
2114
2178
2181
2182
2176
2056
2179
2115
2175
2053
2083
2091
Processings site no 2090
Developed, Low Intensity - Includes areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 20-49 percent of total cover.  These areas most commonly include single-family housing units.
no valid overpasses
2085
2030
2084
2026
2214
2215
2189
2190
2187
2183
2191
2192
2185
2184
2218
2149
2217
2186
2219
2197
2017
2051
2012
2009
2027
2013
Processings site no 2031
Cultivated Crops - Areas used for the production of annual crops, such as corn, soybeans, vegetables, tobacco, and cotton, and also perennial woody crops such as orchards and vineyards. Crop vegetation accounts for greater than 20 percent of total vegetation. This class also includes all land being actively tilled.
Processing 3 sentinel overapasses
0.0 % 
processing PRISM
0.0 % 
processing MODIS
0.0 % 
Processing Landsat
0.0 % 
3.546 % 
7.092 % 
10.63 % 
14.18 % 
17.73 % 

/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:184: RuntimeWarning: Mean of empty slice
/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:187: RuntimeWarning: Mean of empty slice


No non-rainy overpaasses 
Processings site no 2092
Cultivated Crops - Areas used for the production of annual crops, such as corn, soybeans, vegetables, tobacco, and cotton, and also perennial woody crops such as orchards and vineyards. Crop vegetation accounts for greater than 20 percent of total vegetation. This class also includes all land being actively tilled.
no valid overpasses
Processings site no 2094
Grassland/Herbaceous - Areas dominated by grammanoid or herbaceous vegetation, generally greater than 80% of total vegetation.  These areas are not subject to intensive management such as tilling, but can be utilized for grazing.
no valid overpasses
Processings site no 2147
Deciduous Forest  - Areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75 percent of the tree species shed foliage simultaneously in response to seasonal change.
no valid overpasses
Processings site no 2093
Grassland/Herbaceous - Areas domina

Pasture/Hay - Areas of grasses, legumes, or grass-legume mixtures planted for livestock grazing or the production of seed or hay crops, typically on a perennial cycle. Pasture/hay vegetation accounts for greater than 20 percent of total vegetation.
no valid overpasses
2076
2077
2075
2207
2006
2206
Processings site no 2201
Shrub/Scrub - Areas dominated by shrubs less than 5 meters tall with shrub canopy typically greater than 20% of total vegetation. This class includes true shrubs, young trees in an early successional stage or trees stunted from environmental conditions.
no valid overpasses
2106
2105
Processings site no 2228
no valid soil moisture data for 2228
2016
Processings site no 2104
Grassland/Herbaceous - Areas dominated by grammanoid or herbaceous vegetation, generally greater than 80% of total vegetation.  These areas are not subject to intensive management such as tilling, but can be utilized for grazing.
no valid overpasses
Processings site no 2199
Pasture/Hay - Areas of gr

In [58]:
t = pd.concat(out_dict)

In [59]:
len(t)

7684

In [61]:
t.to_csv("../data/all_data_descending.csv")

In [63]:
len(t.dropna())

5735

In [64]:
t

sigma std_sigma precip  LAI   rzsm     psi_rz  \
     date                                                                   
2005 2016-02-04 11:55:00 -7.57724   4.53272    0.0  0.5  65.38  0.0127681   
2006 2015-02-20 12:53:00 -4.94962    2.9613    0.0  0.3  23.16    66.3272   
     2015-03-04 12:53:00 -5.13477   3.19202    0.0  0.3  22.88    74.1262   
     2015-03-28 12:53:00 -8.79918   5.19831    0.0  0.4   23.7    53.7287   
     2015-07-26 12:53:00 -10.7665   6.26717    0.0  0.6   44.7   0.162826   
     2015-11-23 12:54:00  -7.9262   4.57906    0.0  0.4  41.14    0.34765   
     2015-12-17 12:54:00 -10.2371   5.91214    0.0  0.3  41.06    0.35389   
     2016-01-10 12:54:00 -3.89937   2.29537    0.0  0.3  41.32   0.334051   
     2016-02-03 12:54:00 -8.76869   5.13395    0.0  0.3  41.06    0.35389   
     2016-02-03 12:54:00 -8.78368   5.12196    0.0  0.3  41.06    0.35389   
     2016-02-27 12:54:00 -6.30812   3.70762    0.0  0.7  40.42    0.40853   
     2016-03-22 12:54:00 -8.00216   4.62771    0.0    1  40.24   0.425539   
     2016-04-15 12:54:00 -11.6784   6.77195    0.0  1.4  40.24   0.425539   
     2016-05-09 12:54:00 -10.3338   5.96976    0.0  0.9  39.92   0.457752   
     2016-07-20 12:53:00 -9.22419   5.58911    0.0  0.4  21.22    147.547   
     2016-08-13 12:53:00 -7.66978   4.68515    0.0  0.5  21.44    134.274   
     2016-09-06 12:53:00 -7.00352    4.1098    0.0  1.5  21.38    137.758   
     2016-09-30 12:54:00 -8.56365   5.13648    0.0  0.5  21.28    143.789   
     2016-10-24 12:54:00 -11.6697   6.76963    0.0  0.4  20.76    180.269   
     2016-11-17 12:54:00 -5.83687     3.512    0.0  0.3  20.42    209.637   
     2016-12-11 12:54:00 -6.46902   3.95921    0.0  0.3  19.64    299.265   
     2017-01-04 12:54:00 -8.02489   4.66324    0.0  0.1   19.1    386.116   
     2017-01-28 12:54:00 -7.35869   4.25087    0.0  0.3   19.1    386.116   
     2017-02-09 12:54:00 -8.04906   4.70508    0.0  0.3  19.08    389.831   
     2017-02-21 12:54:00 -6.56368   3.80832    0.0  0.6  19.26    357.772   
     2017-03-05 12:54:00 -8.28748   4.93349    0.0  0.5  19.34    344.472   
     2017-03-17 12:54:00 -9.37563   5.46725    0.0  0.7  19.44    328.612   
     2017-04-10 12:54:00 -10.6642   6.20083    0.0  0.9   19.9    265.372   
     2017-04-22 12:54:00 -12.2626   7.13236    0.0  1.2  20.28    223.241   
     2017-05-04 12:54:00 -5.47152   3.16328    0.0  0.8   20.2    231.453   
...                           ...       ...    ...  ...    ...        ...   
808  2018-01-24 13:32:00 -16.1565   9.33363    0.0  0.1   6.36    90.2992   
     2018-02-04 13:39:00  -12.837   7.42266    0.0  0.1   5.98    113.213   
     2018-02-16 13:39:00  -12.187   7.07945    0.0    0    5.9    118.951   
     2018-02-17 13:32:00 -18.2511   10.6136    0.0    0   5.82    125.064   
     2018-02-28 13:39:00 -11.8517   6.90993    0.0    0    6.2    99.1521   
     2018-03-01 13:32:00 -12.3853   7.24543    0.0  0.1   6.44    86.2495   
     2018-03-12 13:39:00  -10.837   6.26597    0.0  0.2   6.54    81.5065   
     2018-03-13 13:32:00 -15.2794   8.82188    0.0  0.2   6.56    80.5981   
     2018-04-29 13:39:00 -11.2464   6.60946    0.0  0.4   6.74    72.9746   
     2018-06-04 13:39:00 -10.4958   6.14913    0.0  0.7   7.72    44.3359   
     2018-06-05 13:32:00 -9.68417   5.63011    0.0  0.7   7.74    43.9168   
     2018-06-28 13:39:00 -11.3215   6.62879    0.0    1   9.26    22.7401   
     2018-07-23 13:32:00 -12.9776     7.634    0.0  0.6   9.18     23.476   
     2018-08-03 13:39:00 -11.3738   6.62748    0.0  0.6   9.02    25.0411   
     2018-08-04 13:32:00 -10.1161   5.87309    0.0  0.4   9.24    22.9213   
     2018-08-15 13:39:00 -15.5982   9.02052    0.0  0.3   9.12    24.0479   
     2018-08-16 13:32:00 -14.1329     8.233    0.0  0.4   8.96    25.6622   
     2018-09-08 13:40:00 -13.5493   7.87648    0.0  0.2   8.74    28.1141   
     2018-09-09 13:32:00 -17.1823   9.93613    0.0  0.2   8.74    28.1141   
     2018-09-21 13:32:00

In [ ]:
len(t.B4)

In [ ]:
x = np.array(mean_sigmas)
x = x.reshape(x.shape[0])
y = np.array(rzsm)
y = y.reshape(y.shape[0])
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

In [ ]:
plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture 
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], "10"))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(mean_sigmas, rzsm, label = 'rzsm')
plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# as a timeseries

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''$\sigma$ vs Soil moisture Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:red'
ax1.set_xlabel('date')
ax1.set_ylabel("$\sigma$ (dB)", color=color)
ax1.plot(dates, mean_sigmas, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# remove the days with high preceding rainfall 

In [ ]:
imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

In [ ]:
def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

In [ ]:
rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])

In [ ]:
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])

In [ ]:
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''2 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("Precip (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# Let's test with 4 day precip accumulation

imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-4, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)
    

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''4 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("3 day precipitation (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation (4 days)
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()